<a href="https://colab.research.google.com/github/FarmIA/farmia/blob/main/model_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'new-plant-diseases-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F78313%2F182633%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T213432Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0e7d7ac286d0d481431c3a4124ff4ba41c0533a5ae63c6e59828a3110ce0fa4980b7e61d9597aee8110937692012e478f24d0c43d0c19e120a0e358c5c3ab4e22656073c2aaf590a0cb94c1cd5ac49faa51e7968ef4b317b21492ce226080740338018af32ca1e5be339bf664ed279c64d1b59e67d41dbb4188f13bd8a427273e3722929a0199fd0670d78f3704c1802c053d7aaa2b5bb2076a198b3d184837c2f802fee912525c2293fd8bc7bc7d3354cb840eae8c35621c928832ec356537264bc52a525e26a9a65d2870556f1461bd5fee1f2d1008e3f3b2e9785f75d353b5b8cecd5fd0e3ba5513768e1a8bd3f7ad0dabed04d3cb834de06cb9989116185'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import tensorflow as tf

train_dir = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_dir = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"

# Définition des paramètres
batch_size = 32
img_height = 256
img_width = 256

# Chargement des images depuis le dossier
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  valid_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


# Affichage des classes
class_names = train_ds.class_names
print(class_names)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Obtenir le premier lot de données
images, labels = next(iter(train_ds))

# Imprimer les dimensions du premier lot de données
print("Dimensions des images:", images.shape)
print("Dimensions des étiquettes:", labels.shape)

# Afficher la première image du lot
img = Image.fromarray(images[0].numpy().astype('uint8'))
plt.imshow(img)
plt.axis('off')  # Désactive les axes
plt.show()

In [ ]:
from tensorflow import keras

model = keras.Sequential()
model.add(keras.layers.InputLayer(shape=(256, 256, 3)))

model.add(keras.layers.Conv2D(32,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(32,(3,3),activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D(3,3))

model.add(keras.layers.Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D(3,3))

model.add(keras.layers.Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(keras.layers.MaxPooling2D(3,3))

model.add(keras.layers.Conv2D(256,(3,3),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(256,(3,3),activation="relu",padding="same"))

model.add(keras.layers.Conv2D(512,(5,5),activation="relu",padding="same"))
model.add(keras.layers.Conv2D(512,(5,5),activation="relu",padding="same"))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(1568,activation="relu"))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(38,activation="softmax"))

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.summary()

In [ ]:
# Entraînement du modèle

from tensorflow.keras import callbacks

# Fonction de callback
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,
    patience=5,
    restore_best_weights=True,
)

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    verbose=1,
                   callbacks=[early_stopping])

# Visualisation de l'historique de l'entraînement
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
model.save("mon_modele_entrene.keras")

In [ ]:
import numpy as np

# Charger une nouvelle image
image_path = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Blueberry___healthy/067e7729-ebb3-4824-80dc-9ceda52f47b8___RS_HL 5388.JPG"
img = Image.open(image_path)

# Prétraiter l'image
img_array = np.array(img)  # Convertir l'image en un tableau numpy
img_array = img_array / 255.0  # Normaliser les valeurs des pixels entre 0 et 1
img_array = np.expand_dims(img_array, axis=0)  # Ajouter une dimension supplémentaire pour correspondre à l'entrée attendue du modèle

# Faire une prédiction avec le modèle
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions)

# Obtenir le nom de la classe prédite (à remplacer par vos classes spécifiques)
predicted_class_name = class_names[predicted_class_index]

# Afficher l'image avec le titre de la classe prédite et la probabilité associée
plt.imshow(img)
plt.title(f"Classe prédite : {predicted_class_name}")
plt.axis('off')
plt.show()


In [ ]:
import os
import numpy as np

test_dir = "/kaggle/input/new-plant-diseases-dataset/test/test"

for filename in os.listdir(test_dir):
    img_path = os.path.join(test_dir, filename)

    # Charger l'image et la prétraiter
    img = Image.open(img_path)
    img_array = np.array(img) / 255.0  # Normaliser les valeurs des pixels

    # Ajouter une dimension supplémentaire pour correspondre à l'entrée attendue du modèle
    img_array = np.expand_dims(img_array, axis=0)

    # Faire une prédiction avec le modèle
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    predicted_probability = predictions[0][predicted_class_index]

    # Obtenir le nom de la classe prédite
    predicted_class_name = class_names[predicted_class_index]

    # Afficher l'image avec le titre de la classe prédite et la probabilité associée
    print(f"Image : {filename}\nPrédiction : {predicted_class_name} (Probabilité : {predicted_probability:.2f})")

In [ ]:
# Charger le modèle externe
import tensorflow as tf
model = tf.keras.models.load_model("/kaggle/working/mon_modele_entrene.keras")

# Examiner les poids du modèle
for layer in model.layers:
    print("Layer Name:", layer.name)
    print("Weights:")
    for weight in layer.weights:
        print(weight.shape)  # Afficher la forme des poids de la couche
    print()
